In [1]:
import pyspark
import pandas as pd
import numpy as np
import pyspark.sql.functions as psf
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_extract, regexp_replace
from pydataset import data
from pyspark.sql.functions import col, expr

# Exercises

1. Create a spark data frame that contains your favorite programming languages.
- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/30 11:41:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
np.random.seed(13)

pandas_dataframe = pd.DataFrame(
    dict(language= ['python', 'sql', 'spark'] * 10)
)
pandas_dataframe.head()

,language
0,python
1,sql
2,spark
3,python
4,sql


In [4]:
df = spark.createDataFrame(pandas_dataframe)
df.show(5)

+--------+
|language|
+--------+
|  python|
|     sql|
|   spark|
|  python|
|     sql|
+--------+
only showing top 5 rows



In [5]:
df.printSchema()



root
 |-- language: string (nullable = true)



In [6]:
df.count(),len(df.columns)


(30, 1)

In [7]:
df.show(5)


+--------+
|language|
+--------+
|  python|
|     sql|
|   spark|
|  python|
|     sql|
+--------+
only showing top 5 rows



2. Load the mpg dataset as a spark dataframe.
A. Create 1 column of output that contains a message like the one below:

  The 1999 audi a4 has a 4 cylinder engine.

For each vehicle.

B. Transform the trans column so that it only contains either manual

In [8]:
df = data('mpg')
df = spark.createDataFrame(df)
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [9]:
df = df.withColumn('new', F.concat(psf.lit('The '), df.year, psf.lit(' '), df.manufacturer, psf.lit(' has a '), df.cyl, psf.lit(' cylinder engine.')))


In [10]:
df.show(5, truncate = False)


+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------------------------+
|manufacturer|model|displ|year|cyl|trans     |drv|cty|hwy|fl |class  |new                                   |
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------------------------+
|audi        |a4   |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi has a 4 cylinder engine.|
|audi        |a4   |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008 audi has a 4 cylinder engine.|
|audi        |a4   |2.8  |1999|6  |auto(l5)  |f  |16 |26 |p  |compact|The 1999 audi has a 6 cylinder engine.|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------------------------+
only showi

In [11]:
df = df.withColumn("trans", regexp_replace(df["trans"], r"\((.*?)\)", ""))


In [12]:
df.show(5)



+------------+-----+-----+----+---+------+---+---+---+---+-------+--------------------+
|manufacturer|model|displ|year|cyl| trans|drv|cty|hwy| fl|  class|                 new|
+------------+-----+-----+----+---+------+---+---+---+---+-------+--------------------+
|        audi|   a4|  1.8|1999|  4|  auto|  f| 18| 29|  p|compact|The 1999 audi has...|
|        audi|   a4|  1.8|1999|  4|manual|  f| 21| 29|  p|compact|The 1999 audi has...|
|        audi|   a4|  2.0|2008|  4|manual|  f| 20| 31|  p|compact|The 2008 audi has...|
|        audi|   a4|  2.0|2008|  4|  auto|  f| 21| 30|  p|compact|The 2008 audi has...|
|        audi|   a4|  2.8|1999|  6|  auto|  f| 16| 26|  p|compact|The 1999 audi has...|
+------------+-----+-----+----+---+------+---+---+---+---+-------+--------------------+
only showing top 5 rows



# 3. Load the tips dataset as a spark dataframe.
A. What percentage of observations are smokers?
B. Create a column that contains the tip percentage
C. Calculate the average tip percentage for each combination of sex and smoker.

In [13]:
df = data('tips')
df = spark.createDataFrame(df)
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [14]:
df.filter(df.smoker == 'Yes').count() / df.count() * 100


38.114754098360656

In [15]:
df = df.withColumn('tip_percent', df.tip / df.total_bill * 100)
df.show(5)

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|       tip_percent|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
+----------+----+------+------+---+------+----+------------------+
only showing top 5 rows

